In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy import stats
import plotly.offline
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import cufflinks as cf

In [2]:
cf.go_offline()
cf.set_config_file(world_readable=True, theme='white')

In [3]:
class UserGenerator:
    def __init__(self):
        self.beta = {}
        self.beta['A'] = np.array([-4, -0.1, -3, 0.1])
        self.beta['B'] = np.array([-6, -0.1, 1, 0.1])
        self.beta['C'] = np.array([2, 0.1, 1, -0.1])
        self.beta['D'] = np.array([4, 0.1, -3, -0.2])
        self.beta['E'] = np.array([-0.1, 0, 0.5, -0.01])
        self.context = None
    def logistic(self, beta, context):
        f = np.dot(beta, context)
        p = 1 / (1 + np.exp(-f))
        return p
    def display_ad(self, ad):
        if ad in ['A', 'B', 'C', 'D', 'E']:
            p = self.logistic(self.beta[ad], self.context)
            reward = np.random.binomial(n=1, p=p)
            return reward
        else:
            raise Exception('Unknown ad!')
    def generate_user_with_context(self):
        location = np.random.binomial(n=1, p=0.6)
        device = np.random.binomial(n=1, p=0.8)
        age = 10 + int(np.random.beta(2, 3) * 60)
        
        self.context = [1, device, location, age]
        return self.context
def get_scatter(x, y, name, showlegend):
    dashmap = {'A': 'solid', 'B': 'dot', 'C': 'dash', 'D': 'dashdot', 'E': 'longdash'}
    s = go.Scatter(x=x, y=y, legendgroup=name, showlegend=showlegend, name=name, line=dict(color='blue', dash=dashmap[name]))
    return s
def visualize_bandits(ug):
    ad_list = 'ABCDE'
    ages = np.linspace(10, 70)
    fig = make_subplots(rows=2, cols=2, subplot_titles=("Desktop International", "Desktop U.S.", "Mobile International", "Mobile U.S."))
    for device in [0, 1]:
        for loc in [0, 1]:
            showlegend = (device == 0) & (loc == 0)
            for ad in ad_list:
                probs = [ug.logistic(ug.beta[ad], [1, device, loc, age]) for age in ages]
                fig.add_trace(get_scatter(ages, probs, ad, showlegend), row=device+1, col=loc+1)
    fig.update_layout(template="presentation")
    fig.show()

In [4]:
ug = UserGenerator()
visualize_bandits(ug)

In [5]:
class RegularizedLR:
    def __init__(self, name, alpha, rlambda, n_dim):
        self.name = name
        self.alpha = alpha
        self.rlambda = rlambda
        self.n_dim = n_dim
        self.m = np.zeros(n_dim)
        self.q = np.ones(n_dim) * rlambda
        self.w = self.get_sampled_weights()
        
    def get_sampled_weights(self):
        w = np.random.normal(self.m, self.alpha * self.q**(-1/2))
        return w
    
    def loss(self, w, *args):
        X, y = args
        n = len(y)
        regularizer = 0.5 * np.dot(self.q, (w - self.m)**2)
        pred_loss = sum([np.log(1 + np.exp(np.dot(w, X[j]))) - y[j] * np.dot(w, X[j]) for j in range(n)])
        return regularizer + pred_loss
    
    def fit(self, X, y):
        if y:
            X = np.array(X)
            y = np.array(y)
            minimization = minimize(self.loss, self.w, args=(X, y), method="L-BFGS-B", bounds=[(-10, 10)]*3 + [(-1, 1)], options={'maxiter': 50})
            self.w = minimization.x
            self.m = self.w
            p = (1 + np.exp(-np.matmul(self.w, X.T))**(-1))
            self.q = self.q + np.matmul(p * (1-p), X**2)
    def calc_sigmoid(self, w, context):
        return 1 / (1 + np.exp(-np.dot(w, context)))
    
    def get_ucb(self, context):
        pred = self.calc_sigmoid(self.m, context)
        confidence = self.alpha * np.sqrt(np.sum(np.divide(np.array(context)**2, self.q)))
        ucb =